In [1]:
import torch, random, math, time, sys, os, tqdm
import numpy as np
import numba

In [77]:
from trainer.classification import Trainer
from trainer.attention_approx import Trainer as ApproxTrainer
batch_size = 8

trainer = Trainer(batch_size=batch_size)
trainer.load()
trainer.model.eval()
bert = trainer.model.bert
fc = trainer.model.classifier
batch = trainer.get_batch()
test_batch = trainer.get_batch(test=False)

approx_trainer = ApproxTrainer(batch_size=batch_size, factor=16)
approx_trainer.load()
approx_bert = approx_trainer.bert
approx_bert = approx_bert.eval()

100%|██████████| 7600/7600 [00:00<00:00, 211094.26it/s]


Classification Dataset Stat.: name:AG_NEWS, nclass:5, max_len:1012, avg_len:236.477525, count:120000
Classification Dataset Stat.: name:AG_NEWS, nclass:5, max_len:892, avg_len:235.2992105263158, count:7600


Some weights of the model checkpoint at google/bert_uncased_L-4_H-256_A-4 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Trainer.__init__: Model initialized. model = bert
Trainer.load: Loading...


100%|██████████| 7600/7600 [00:00<00:00, 200019.52it/s]


Classification Dataset Stat.: name:AG_NEWS, nclass:5, max_len:1012, avg_len:236.477525, count:120000
Classification Dataset Stat.: name:AG_NEWS, nclass:5, max_len:892, avg_len:235.2992105263158, count:7600


Some weights of the model checkpoint at google/bert_uncased_L-4_H-256_A-4 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Trainer.__init__: Model initialized. model = bert
Trainer.load: Loading...


In [78]:
import importlib
import models.sparse_token as sparse
importlib.reload(sparse)

<module 'models.sparse_token' from 'f:\\Library\\discrete_edge_learning\\models\\sparse_token.py'>

In [79]:
sparse_bert = sparse.SparseBertModel(bert.config)
sparse_bert.to(trainer.device)
sparse_bert.eval()
sparse_bert.load_state_dict(bert.state_dict())

<All keys matched successfully>

In [80]:
def eval_fc(lm_output, fc=fc, batch=batch):
    last_hidden = lm_output.last_hidden_state[:,0,:]
    x = fc(last_hidden)
    return torch.argmax(x, dim=-1), batch.labels, lm_output

def eval(bert, fc=fc, batch=batch):
    lm_output = bert(
        input_ids = batch.input_ids, 
        attention_mask = batch.attention_masks,
        output_hidden_states = True,
        output_attentions = True,
    )
    return eval_fc(lm_output, fc=fc, batch=batch)

def approx_eval(sparse_bert, approx_bert, fc=fc, batch=batch):
    lm_output = sparse.run_bert_with_approx(
        sparse_bert, 
        approx_bert, 
        {
            'input_ids': batch.input_ids,
            'attention_mask': batch.attention_masks,
            'output_hidden_states': True,
            'output_attentions': True,
        }
    )
    return eval_fc(lm_output, fc=fc, batch=batch)
    
eval(bert)[:2], approx_eval(sparse_bert, approx_bert)[:2]

apply input mask, before softmax. input_mask:torch.Size([8, 71]), attention_scores:torch.Size([8, 4, 71, 71])
SelfAttention.forward: last_attention_probs backuped
apply output mask. mask:torch.Size([8, 71, 1]) context:torch.Size([8, 71, 256])
apply input mask, before softmax. input_mask:torch.Size([8, 71]), attention_scores:torch.Size([8, 4, 71, 71])
SelfAttention.forward: last_attention_probs backuped
apply output mask. mask:torch.Size([8, 71, 1]) context:torch.Size([8, 71, 256])
apply input mask, before softmax. input_mask:torch.Size([8, 71]), attention_scores:torch.Size([8, 4, 71, 71])
SelfAttention.forward: last_attention_probs backuped
apply output mask. mask:torch.Size([8, 71, 1]) context:torch.Size([8, 71, 256])
apply input mask, before softmax. input_mask:torch.Size([8, 71]), attention_scores:torch.Size([8, 4, 71, 71])
SelfAttention.forward: last_attention_probs backuped
apply output mask. mask:torch.Size([8, 71, 1]) context:torch.Size([8, 71, 256])


((tensor([1, 2, 4, 1, 3, 4, 3, 3], device='cuda:0'),
  tensor([1, 2, 4, 1, 3, 4, 3, 3], device='cuda:0')),
 (tensor([1, 2, 4, 1, 3, 4, 3, 3], device='cuda:0'),
  tensor([1, 2, 4, 1, 3, 4, 3, 3], device='cuda:0')))

In [109]:
importlib.reload(sparse)
def accuracy(batch_eval, N=50):
    trainer.seed()
    trainer.dataset.batch_size = 128
    acc_sum = 0
    for i in tqdm.tqdm(range(N)):
        batch = trainer.get_batch(test=True)
        with torch.no_grad():
            output, label, _ = batch_eval(batch)
        acc_sum += torch.mean((output == label) * 1.0)
    return acc_sum.item() / N

# setup for evaluation
sparse_bert = sparse.SparseBertModel(bert.config)
sparse_bert.to(trainer.device)
sparse_bert.eval()
sparse_bert.load_state_dict(bert.state_dict())
sparse.set_print(sparse_bert, False)
sparse.set_backup_last_inputs(sparse_bert, False)
sparse.set_output_masking(sparse_bert, False)

sparse_bert = sparse_bert.to(trainer.device)
approx_bert = approx_bert.to(trainer.device)
bert = bert.to(trainer.device)
sparse.set_print(sparse_bert, False)
sparse.set_backup_last_inputs(sparse_bert, False)
sparse.set_output_masking(sparse_bert, False)

acc_approx = accuracy(lambda batch: approx_eval(sparse_bert, approx_bert, batch=batch))
acc_bert = accuracy(lambda batch: eval(bert, batch=batch))
acc_bert, acc_approx
#(0.934375, 0.92203125)
#(0.934375, 0.93359375)

100%|██████████| 50/50 [00:01<00:00, 37.71it/s]


(0.934375, 0.93359375)

In [87]:
def benchmark(eval, batch_size=8, N=100, WARM=20, amp=False, device=trainer.device):
    assert WARM < (N * 0.33)
    trainer.dataset.batch_size = batch_size
    batch = trainer.get_batch(test=True)
    batch = batch.to(device)
    assert batch.input_ids.shape[0] == batch_size
    for i in tqdm.tqdm(range(N)):
        if i == WARM: t = time.time()
        with torch.no_grad(), torch.cuda.amp.autocast(enabled=amp):
            eval(batch)
    t = time.time() - t
    t_item = t / (batch_size * (N-WARM))
    return t, t_item * 1000, 1.0/t_item

In [107]:
importlib.reload(sparse)
sparse.timer_reset()
benchmark_device = 'cpu'

sparse_bert = sparse.SparseBertModel(bert.config)
sparse_bert.to(benchmark_device)
sparse_bert.eval()
sparse_bert.load_state_dict(bert.state_dict())
sparse.set_print(sparse_bert, False)
sparse.set_backup_last_inputs(sparse_bert, False)
sparse.set_output_masking(sparse_bert, False)

sparse_bert=sparse_bert.to(benchmark_device)
approx_bert=approx_bert.to(benchmark_device)
time_approx = benchmark(
    eval = lambda batch: approx_eval(sparse_bert, approx_bert, batch=batch, fc=lambda x: x),
    batch_size = 32,
    WARM = 1,
    N = 25,
    device = benchmark_device,
)
time_approx, sparse.__timer_cum

 20%|██        | 5/25 [00:04<00:16,  1.23it/s]


KeyboardInterrupt: 

In [89]:
bert = bert.to(benchmark_device)
time_bert = benchmark(
    lambda batch: eval(bert, batch=batch, fc=lambda x: x), 
    batch_size = 32,
    WARM = 1,
    N=25,
    device = benchmark_device
)
time_bert

100%|██████████| 25/25 [00:06<00:00,  3.93it/s]


(6.103064775466919, 7.946698926389217, 125.83841532981987)

TypeError: topk(): argument 'k' (position 2) must be int, not list